In [2]:
import pandas as pd
import snowflake.connector
from collections import defaultdict
import json
import re

In [3]:
con = snowflake.connector.connect(user='vishal.kumar@scale.com',
                                 account='pxa65918',
                                 authenticator='externalbrowser',
                                 warehouse='COMPUTE_WH',
                                 database='SCALE_CRAWLER',
                                 role='GENERAL_RO')

cs = con.cursor()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


In [4]:
sql = '''
with cf as (
SELECT
    DISTINCT CAST(product_id as varchar) as PRODUCT_ID,
    CAST(catalog_id as varchar) as CATALOG_ID,
    parse_json(attribute_labels):title[0] as title_score,
    parse_json(attribute_labels):image[0] as media_score,
    parse_json(attribute_labels):description[0] as description_score,
    parse_json(attribute_labels):material[0] as material_score,
    parse_json(attribute_labels):brand[0] as brand_score,
    parse_json(attribute_labels):color[0] as color_score,
    parse_json(attribute_labels):dimensions[0] as dimensions_score,
    parse_json(attribute_labels):gemstone[0] as gemstone_score,
    parse_json(attribute_labels):package_quantity[0] as qty_score,
    parse_json(attribute_labels):pattern[0] as pattern_score,
    parse_json(attribute_labels):product_height[0] as height_score,
    parse_json(attribute_labels):product_length[0] as length_score,
    parse_json(attribute_labels):product_volume[0] as volume_score,
    parse_json(attribute_labels):product_width[0] as width_score,
    parse_json(attribute_labels):scent[0] as scent_score,
    parse_json(attribute_labels):size[0] as size_score,
    parse_json(attribute_labels):title as Title,
    parse_json(attribute_labels):image as Media,
    parse_json(attribute_labels):description as Description,
    parse_json(attribute_labels):brand as Brand,
    parse_json(attribute_labels):color as Color,
    parse_json(attribute_labels):dimensions as Dimensions,
    parse_json(attribute_labels):gemstone as Gemstone,
    parse_json(attribute_labels):material as Material,
    parse_json(attribute_labels):package_quantity as Package_Quantity,
    parse_json(attribute_labels):pattern as Pattern,
    parse_json(attribute_labels):product_height as Product_Height,
    parse_json(attribute_labels):product_length as Product_Length,
    parse_json(attribute_labels):product_width as Product_Width,
    parse_json(attribute_labels):scent as Scent,
    parse_json(attribute_labels):size as Size,
    parse_json(attribute_labels):volume as Volume,
    TO_DATE(date_received) as date_received
FROM PC_FIVETRAN_DB.GSHEETS.flamingocfapril_25
)

SELECT 
DISTINCT
CI.PRODUCT_ID,
CI.CATALOG_ID,
title,
media,
description,
material,
scent,
Color,
pattern,
Gemstone,
Brand,
product_length,
product_width,
product_height,
volume,
Dimensions,
size,
package_quantity,
title_score,
media_score,
description_score,
material_score,
scent_score,
color_score,
pattern_score,
gemstone_score,
brand_score,
length_score,
width_score,
height_score,
volume_score,
dimensions_score,
size_score,
qty_score,
date_received
FROM  scale_crawler.PUBLIC.CUSTOMERINPUTS as CI 
JOIN cf ON cf.PRODUCT_ID = CI.PRODUCT_ID
'''
cs.execute(sql)

custdf = cs.fetch_pandas_all()

In [5]:
sql = '''
select CATALOG_ID, PRODUCT_ID from PUBLIC.CUSTOMERINPUTS
'''
cs.execute(sql)

mapdf = cs.fetch_pandas_all()

In [6]:
print(len(custdf))
print(len(mapdf))

244225
42222871


In [7]:
subdf = pd.read_csv('Submissions.csv', dtype = {'PID': str, 'CID': int})

In [8]:
subdf['CATALOG_ID']=""

In [9]:
mergedf = subdf.merge(mapdf, on='PRODUCT_ID', how='left')

In [10]:
mergedf['CATALOG_ID_y'].isna().sum()

83260

In [11]:
print((len(mergedf)))

119044


In [12]:
mergedf

,PRODUCT_ID,CATALOG_ID_x,CATALOG_ID_y
0,4045691762205850,,NaN
1,5469583513082030,,NaN
2,4514081515366900,,NaN
3,5466600043350370,,NaN
4,2750747575040950,,NaN
...,...,...,...
119039,8065551086795646,,200412037453681
119040,3959582447412695,,200412037453681
119041,4582449348456743,,200412037453681
119042,5010260559022993,,200412037453681


In [13]:
mergedf.to_csv('merged2.csv')